In [6]:
import gym
import torch

from collections import deque
import random

import copy
from torch.autograd import Variable
import numpy as np
import time
import json
import os
from datetime import datetime
import psutil
import os

In [7]:
# Get current process ID
pid = os.getpid()
process = psutil.Process(pid)
env = gym.envs.make("LunarLander-v2",render_mode="human")

/home/janak/Documents/Pytorch_CPU/venv/lib/python3.10/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/janak/Documents/Pytorch_CPU/venv/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [8]:
class DQN_C():
    def __init__(self, n_state, n_action, n_hidden=50, lr=0.05):
        self.criterion = torch.nn.MSELoss()
        self.model = torch.nn.Sequential(
                        torch.nn.Linear(n_state, n_hidden),
                        torch.nn.ReLU(),
                        torch.nn.Linear(n_hidden, n_hidden),
                        torch.nn.ReLU(),
                        torch.nn.Linear(n_hidden, n_hidden),
                        torch.nn.ReLU(),
                        torch.nn.Linear(n_hidden, n_action)
                )


        self.model_target = copy.deepcopy(self.model)

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)

    def update(self, s, y):
        """
        Update the weights of the DQN given a training sample
        @param s: state
        @param y: target value
        """
        s = np.array(s)
        y_pred = self.model(torch.Tensor(s))

        loss = self.criterion(y_pred, Variable(torch.Tensor(np.array(y))))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


    def predict(self, s):
        """
        Compute the Q values of the state for all actions using the learning model
        @param s: input state
        @return: Q values of the state for all actions
        """
        with torch.no_grad():
            return self.model(torch.Tensor(np.array(s)))

    def target_predict(self, s):
        """
        Compute the Q values of the state for all actions using the target network
        @param s: input state
        @return: targeted Q values of the state for all actions
        """
        with torch.no_grad():
            return self.model_target(torch.Tensor(np.array(s)))

    def replay(self, memory, replay_size, gamma):
        """
        Experience replay with target network
        @param memory: a list of experience
        @param replay_size: the number of samples we use to update the model each time
        @param gamma: the discount factor
        """
        if len(memory) >= replay_size:
            replay_data = random.sample(memory, replay_size)

            states = []
            td_targets = []
            for state, action, next_state, reward, is_done in replay_data:
                states.append(state)
                q_values = self.predict(state).tolist()
                if is_done:
                    q_values[action] = reward
                else:
                    q_values_next = self.target_predict(next_state).detach()

                    q_values[action] = reward + gamma * torch.max(q_values_next).item()

                td_targets.append(q_values)

            self.update(states, td_targets)

    def copy_target(self):
        self.model_target.load_state_dict(self.model.state_dict())

    def name(self):
      return "Model C"

In [9]:
target_update = 10
n_episode = 1000
replay_size = 142
n_state = env.observation_space.shape[0]
n_action = env.action_space.n
n_hidden = 75
lr = 0.001

dqn = DQN_C(n_state,n_action,n_hidden,lr)

#load model
# model_path = '/home/janak/Documents/Pytorch_CPU/lunar_working_2024-04-04-18-28.pth'
# model_path = '2024-04-05 23:35:11.688730.pth'
model_path = '/home/janak/Documents/Pytorch_CPU/Lunar_Plots/2024-04-06 02:41:46.052699.pth'
dqn.model.load_state_dict(torch.load(model_path))

print(dqn.model)

count = 0
max_reward = 0
episodes = 10

Sequential(
  (0): Linear(in_features=8, out_features=75, bias=True)
  (1): ReLU()
  (2): Linear(in_features=75, out_features=75, bias=True)
  (3): ReLU()
  (4): Linear(in_features=75, out_features=75, bias=True)
  (5): ReLU()
  (6): Linear(in_features=75, out_features=4, bias=True)
)


In [10]:
for i in range(episodes):
    total_reward = 0
    
    state = env.reset()
    is_done = False
    while not is_done:
        q_values = dqn.predict(state)
        # print(q_values)
        action = torch.argmax(q_values).item()
        next_state,reward,is_done,_ = env.step(action)
        total_reward += reward
        state = next_state
    
    print(f"Total reward: {total_reward}")
    if total_reward > max_reward:
        max_reward = total_reward
    if total_reward>200:
        count += 1
    time.sleep(2)

print((count/episodes)*100)
print(max_reward)
# Close the environment when done
env.close()

Total reward: 4.500164346242386
Total reward: -68.75175159331314
Total reward: 275.81825064728537
Total reward: 220.52200391012263
Total reward: -77.83496847204154
Total reward: -58.17361489870757
Total reward: 227.21278705071256
Total reward: 194.7350565492164
Total reward: 171.62243572523968
Total reward: 240.96052491094287
40.0
275.81825064728537
